In [1]:
import warnings
warnings.filterwarnings('ignore')
import mediapipe as mp
import cv2
import numpy as np
import pickle

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_draw_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
video_cap = cv2.VideoCapture(0)
with open('model.pkl','rb') as f:
    clf = pickle.load(f)
true_labels = {0: 'B',1: 'A',2: 'C',3: 'D',4: 'F'}
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:
    while video_cap.isOpened():
        ret, frame = video_cap.read()
        H, W, _ = frame.shape
        if not ret:
            print('No frame displayed')
            continue

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        hand_landmarks = hands.process(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        str_label = None
        if hand_landmarks.multi_hand_landmarks:
            for landmark in hand_landmarks.multi_hand_landmarks:
                sample = []
                x_ = []
                y_ = []
                data_aux=[]
                mp_drawing.draw_landmarks(
                    frame,
                    landmark,
                    mp_hands.HAND_CONNECTIONS,
                    mp_draw_styles.get_default_hand_landmarks_style(),
                    mp_draw_styles.get_default_hand_connections_style()
                )
                for i in range(len(landmark.landmark)):
                    x,y,z = landmark.landmark[i].x,landmark.landmark[i].y,landmark.landmark[i].z
                    data_aux.append(x)
                    data_aux.append(y)
                    data_aux.append(z)
                sample.append(data_aux)
                predicted_label = round(clf.predict(np.asarray(sample)).mean())
                str_label = true_labels[predicted_label]
            for i in range(len(landmark.landmark)):
                x = landmark.landmark[i].x
                y = landmark.landmark[i].y

                x_.append(x)
                y_.append(y)

                x = int(np.mean(x_) * W) 
                y = int(np.mean(y_) * H) 
        
            cv2.putText(frame,str_label,(x,y),cv2.FONT_HERSHEY_SIMPLEX,2,(255,205,0),3,cv2.LINE_AA)
        cv2.imshow('Frame', frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
video_cap.release()
cv2.destroyAllWindows()